In [2]:
import uproot
import awkward as ak
import numpy as np
import os
import math
import torch_geometric
import torch
from torch_geometric.data import Data

In [11]:
def analyze(filein,findx):
    try:
        file = uproot.open(filein)
    except:
        print("corrupt file:",filein)
        return 0
    trks =  file["ticlNtuplizer/tracksters"]
    calos = file["ticlNtuplizer/simtrackstersCP"]
    asns = file["ticlNtuplizer/associations"]

    calo = calos.arrays(["stsCP_trackster_barycenter_eta","stsCP_trackster_barycenter_phi","stsCP_barycenter_z"],library="np")
    trk = trks.arrays(["raw_energy","raw_em_energy","barycenter_x","barycenter_y","eVector0_x","eVector0_y","eVector0_z","trackster_barycenter_eta","trackster_barycenter_phi","barycenter_z","id_probabilities"],library="np")
    asn = asns.arrays(["tsCLUE3D_recoToSim_CP","tsCLUE3D_recoToSim_CP_score"],library="np")



    def dr1(v1,v2):
        return np.sqrt((v1-v2)**2)

    nevts = len(trk["raw_energy"])
    thres = 0.1

    for evs in range(nevts): ## loop over events
        #print("evs==================================================>",evs)
        for ncal in range(len(calo["stsCP_barycenter_z"][evs])):
            #print("ncal........................>",ncal)
            #print("ntrks",len(trk["raw_energy"][evs]))
            #print("ens",trk["raw_energy"][evs])
            #print(trk['trackster_barycenter_eta'][evs])
            trklist = []
            for ntrk in range(len(trk["raw_energy"][evs])):
                if dr1(trk["trackster_barycenter_eta"][evs][ntrk],calo["stsCP_trackster_barycenter_eta"][evs][ncal]) < 0.09 and dr1(trk["trackster_barycenter_phi"][evs][ntrk],calo["stsCP_trackster_barycenter_phi"][evs][ncal]) < 0.3 :

                        #print("inside win",ntrk)
                        trklist.append(ntrk)
            #print("trksforcalo:",trklist)

            labeltrklist = []

            featlist= []
            for trks in trklist:
                #print("trks",trks)
                bv = np.array([trk["barycenter_x"][evs][trks],
                                 trk["barycenter_y"][evs][trks],
                                 trk["barycenter_z"][evs][trks]])
                bv = bv /np.linalg.norm(bv)
                pcv = np.array([trk["eVector0_x"][evs][trks],
                                 trk["eVector0_y"][evs][trks],
                                 trk["eVector0_z"][evs][trks]])


                #angle  = (np. dot(bv, pcv))
                angle  = np.arccos(np.dot(bv, pcv))
                if math.isnan(angle)  :
                    angle = 999 

                featlist.append([trk["barycenter_x"][evs][trks],
                                 trk["barycenter_y"][evs][trks],
                                 trk["barycenter_z"][evs][trks],
                                 trk["raw_energy"][evs][trks],
                                 trk["eVector0_x"][evs][trks],
                                 trk["eVector0_y"][evs][trks],
                                 trk["eVector0_z"][evs][trks],
                                angle
                                ])


                cpidx = np.array(asn["tsCLUE3D_recoToSim_CP"][evs][trks])
                cpscore = asn["tsCLUE3D_recoToSim_CP_score"][evs][trks]
                if (np.all(cpscore==1.)):
                    #print("np.all(cpscore==1.)",np.all(cpscore==1.))
                    labeltrklist.append(0)
                else:
                    #labeltrklist.append(cpidx[np.argmin(cpscore)])
                    score = cpscore[np.argwhere(cpidx == ncal)].flatten()
                    #print("ncal,cpidx,cpscore,score",ncal,cpidx,cpscore,score)
                    if score > thres:
                        labeltrklist.append(0)
                        #print(0)
                    else:
                        idproba = (trk["id_probabilities"][evs][trks][0] > 0.5) or  (trk["id_probabilities"][evs][trks][1] > 0.5)
                        enfrac = (trk["raw_em_energy"][evs][trks] / trk["raw_energy"][evs][trks] > 0.9)
                        #if trk["id_probabilities"][evs][trks]
                        #print("prob0,prob1",trk["id_probabilities"][evs][trks][0],trk["id_probabilities"][evs][trks][1])
                        #print("em,raw:",trk["raw_em_energy"][evs][trks],trk["raw_energy"][evs][trks])
                        #print("idproba,enfrac:",idproba,enfrac)
                        if idproba and enfrac:
                            labeltrklist.append(1)
                            #print(1)
                        else:
                            labeltrklist.append(0) 

                    #print(ncal,cpidx,cpscore,score)
                    #print(np.where(cpidx==1))
                #print(cpidx)
                #print(cpscore)
                #print(cpidx[np.argmin(cpscore)])
            #print(labeltrklist)
            labeltrklist = np.array(labeltrklist)
            featlist = np.array(featlist)
            #print(featlist.shape)
            adj = np.ones(shape=(labeltrklist.shape[0],labeltrklist.shape[0]))
            np.fill_diagonal(adj, 0)
            #print(adj)
            rw,cl = np.where(adj>0)
            edgear = np.array([rw,cl])
            edglabel = labeltrklist[rw]*labeltrklist[cl]
            #print("edgear.shape",edgear.shape)
            #print("edglabel.shape",edglabel.shape)
            #print(labeltrklist[rw]*labeltrklist[cl])
            torch.save(Data(x = torch.tensor(featlist, dtype=torch.float32),
            #edge_index=torch.tensor(pairs_sel, dtype=torch.long).t().contiguous()  , 
            #y=torch.tensor(label),
                        edgeidx = torch.tensor(edgear,dtype=torch.int32),
                        edgelabel = torch.tensor(edglabel,dtype=torch.int32),
                        nodelabel = torch.tensor(labeltrklist,dtype=torch.int32)),
            #osp.join(processed_dir, 'datapi_{}_{}.pt'.format(evt,ntk)))
            #osp.join(processed_dir, 'data.pt'.format(evt,ntk)))
            osp.join(processed_dir, 'datapho_{}_{}_{}.pt'.format(findx,evs,ncal)))

        #if evs > 2:
        #    break

In [12]:
import os.path as osp
raw_dir = "/grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/"
listfiles = os.listdir(raw_dir)
listfiles = [raw_dir+i for i in listfiles]
processed_dir = "/grid_mnt/data__data.polcms/cms/sghosh/hackathon/graphsv1/"
#print(listfiles)
if not os.path.exists(processed_dir):
            os.makedirs(processed_dir)

for i,j in enumerate(listfiles):
    print("processing file:",j)
    analyze(j,i)


processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_0.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1000.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1001.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1002.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1003.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1004.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1005.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1006.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1007.root
processing file: /grid_mnt/data__data.po

processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1069.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_106.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1070.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1071.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1072.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1073.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1074.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1075.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1076.root
processing file: /grid_mnt/data__data.

processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1137.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1138.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1139.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_113.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1140.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1141.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1142.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1144.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1145.root
processing file: /grid_mnt/data__data.

processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1202.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1203.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1204.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1205.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1206.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1207.root
corrupt file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1207.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1208.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1209.root
processing file: /grid_mnt/data__data.po

processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1267.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1268.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1269.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_126.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1270.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1271.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1272.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1273.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/new_ntuples_13872541_1274.root
corrupt file: /grid_mnt/data__data.pol

...

processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1000.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1001.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1002.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1003.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1005.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1007.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1009.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1010.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1011.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfil

processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1080.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1081.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1082.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1083.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1084.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1085.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1086.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1087.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1088.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfil

processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1157.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1158.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1159.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_115.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1161.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1162.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1163.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1165.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1167.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfile

processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1240.root
corrupt file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1240.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1241.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1242.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1243.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1244.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1245.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1247.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_1248.root
processing file: /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/

KeyInFileError: not found: 'ticlNtuplizer' (with any cycle number)

    Available keys: (none!)

in file /grid_mnt/data__data.polcms/cms/sghosh/hackathon/rootfiles/ntuples_13845741_124.root

In [ ]:
new_ntuples_13872541_1157